#  Assignment 3 
*  Anirudh Gupta         18ME10006
*  Rahul Kumar Meena     18EC30033
*  Shreyans Ray          18ME10084

##  1. Sentence classification:

In [16]:
                                            ## Reading Data Files##
import pandas as pd
import nltk
import string
from collections import Counter
import numpy as np

df = pd.read_csv("traindata.csv")
df1= pd.read_csv("testdata.csv")
df40 = pd.read_csv("40.csv")
df10= pd.read_csv("10.csv")

df

,category,text,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,science,Outer space is not friendly to life. Extreme t...,NaN,NaN,NaN,NaN
1,sports,"Tennis, original name lawn tennis, game in whi...",NaN,NaN,NaN,NaN
2,business,One woman who frequently flew on Southwest was...,NaN,NaN,NaN,NaN
3,covid,"In December 2019, almost seven years after the...",NaN,NaN,NaN,NaN
4,science,Any life-forms that somehow find themselves in...,NaN,NaN,NaN,NaN
5,science,"As small communities, new research shows, some...",NaN,NaN,NaN,NaN
6,science,Balls of Deinococcus bacteria as thin as five ...,NaN,NaN,NaN,NaN
7,science,They stayed there for three years. Microbes in...,NaN,NaN,NaN,NaN
8,science,The group’s outer layers had shielded them fro...,NaN,NaN,NaN,NaN
9,science,It was known that microbes could survive insid...,NaN,NaN,NaN,NaN


In [17]:
                                   ## Function for filtering the words ##
    
def filtering(ls ):
    
    
    table = str.maketrans('', '', string.punctuation)                   #Removes Punctuation
    stripped = [w.translate(table) for w in ls]
  

    final = [w.lower() for w in stripped]                              #Makes word lowercase
 
   
    from nltk.corpus import stopwords                                  #Removes Stop words
    stop_words = stopwords.words('english')
    
    filtered_sentence = [] 

    for w in final: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
        

    from nltk.stem import PorterStemmer                                #Porter Stemming of words
    FinalSet=[]
    ps = PorterStemmer()
    for i in filtered_sentence:
        trial=ps.stem(i)
        FinalSet.append(trial)
    FinalSet


   
    return FinalSet                                                   #returns Final words


In [18]:
                        ## Calling filtering function and making vocabulary of all words ##
    
n= len(df)
ls=[]
for i in range(n):
    ls+= df.text[i].split()
full_vocab =filtering(ls)                                                #filtering full vocab
full_vocab = list(Counter(full_vocab))                                   #removes duplicates
n_fv =len(full_vocab)
print('Number of words in Full vocab after removing duplicates =',n_fv)


Number of words in Full vocab after removing duplicates = 726


In [19]:
                                              ## prior distribution ##

category = df.category.values
clas =Counter(category)

prior_dist_df1 = pd.DataFrame.from_dict(clas, orient='index')
prior_dist_df = (prior_dist_df1.T+1)/(n+4)                                #Probability using Laplace smoothing

class_label =list(clas)
prior_dist_df1 =prior_dist_df1.T
prior_dist_df                                                             #DataFrame for prior distribution

,science,sports,business,covid
0,0.25,0.25,0.238095,0.261905


In [20]:
                                         ## Making a Vocab for each class ##
    
# This vocab will be used for calculating Class conditional distribution


topic=[]
for i in range(len(class_label)):
    ls1=[]
   
    for j in range(n):
        if df.category[j] ==class_label[i] :            
            ls1 += df.text[j].split()
            
    topic.append(ls1)  
    topic[i] = filtering(topic[i])                # topic[i] conatins vocab of each class 

In [21]:
                                  ## Calculating class conditional distribution ##
    
class_condn = np.zeros((n_fv,len(class_label)))
for i in range(n_fv):    
    for j in range(len(class_label)):
        
        
        nk= topic[j].count(full_vocab[i])                               #keeps track of count each word in each class i
        class_condn[i,j] =(nk+1)/(len(topic[j])+n_fv)                    #prob of each word in class i with laplace smoothing
    class_condn[i,:]= class_condn[i,:]/ np.sum(class_condn[i,:])
       
        
print(class_condn.shape)                           ###class_condn contains 726 rows corresponding to each word and column 
class_condn                                       #represent the class conditional prob of that word respective to class ###

(726, 4)


array([[0.47978953, 0.19056487, 0.10359092, 0.22605468],
       [0.80880494, 0.05840811, 0.06350121, 0.06928574],
       [0.43475306, 0.17267709, 0.18773428, 0.20483558],
       ...,
       [0.22013726, 0.17487013, 0.19011854, 0.41487407],
       [0.22013726, 0.17487013, 0.19011854, 0.41487407],
       [0.22013726, 0.17487013, 0.19011854, 0.41487407]])

In [22]:
                                            ## Predicting Class of Test data ## 
    
    
n1= len(df1)                                                      #df1 is dataframe of testdata         
ls2=[]
pdf= pd.DataFrame([])                                           #empty dataframe to store posterior distribution
pred =[]; max=[] ; test_label=[]
for i in range(n1):
    
    ls2= df1.text[i].split()
    contents = filtering(ls2)                                     #filtering test sentence
    p=np.ones((1,4))
    
    for x in contents:                                            #looks for the words from test sentence in full_vocab
        for j in range(n_fv):
            if x == full_vocab[j]:                              
                p =np.multiply(class_condn[j],p)                 
                break

    p =np.multiply(p,np.array(prior_dist_df[0:]))                #multiplying class conditional prob with prior prob 
    p = p/np.sum(p,axis=1)                                       #converting p to probabiltiy
    
    
    temp = pd.DataFrame(p) 
    pred.append(class_label[np.argmax(p,axis=1)[0]]); max.append(np.max(p,axis=1)[0])
    test_label.append(df1.category[i])
    pdf =pdf.append(temp)              
                                                                ### p is array with 4 values each value of index i represents
                                                                  # posterior probabiltiy###     

In [23]:
                                      ## Dataframe of Results of Prediction##
    
pdf.columns =['prob_science', 'prob_sports','prob_business','prob_covid'] 
pdf['Category'] =test_label                       
pdf['Predicted Class']=pred                                                       
pdf['Prob of Preicted Class'] =max
indes =[i+1 for i in range (n1)]; pdf['index'] =indes
pdf.set_index('index')

,prob_science,prob_sports,prob_business,prob_covid,Category,Predicted Class,Prob of Preicted Class
index,,,,,,,
1,9.996234e-01,2.642789e-05,4.156485e-05,3.085672e-04,science,science,0.999623
2,7.483081e-01,4.449265e-03,1.985094e-01,4.873329e-02,science,science,0.748308
3,9.951086e-01,1.719662e-04,3.528555e-03,1.190888e-03,science,science,0.995109
4,9.999182e-01,4.543284e-06,5.509491e-05,2.213652e-05,science,science,0.999918
5,6.811741e-01,4.268098e-02,5.223620e-02,2.239087e-01,science,science,0.681174
6,6.969953e-09,9.999999e-01,7.923316e-08,7.415949e-10,sports,sports,1.000000
7,2.259883e-09,1.000000e+00,8.853048e-11,2.551702e-10,sports,sports,1.000000
8,4.365876e-08,9.999999e-01,1.431802e-08,6.725256e-08,sports,sports,1.000000
9,1.796961e-02,9.752701e-01,2.367084e-03,4.393163e-03,sports,sports,0.975270


## 2. Sentence Completion:

In [24]:
                                        ## Vocabulary of all words in 40.csv ##
    
n_2=len(df40)
l2=[]
for i in range(n_2):
    l2 += df40.text[i].split()
group = filtering(l2)                                      #List containing all words 
vocab_2=list(Counter(group))                               #removes duplicate from vocab
n_fv2 =len(vocab_2)
print('Number of Words in Vocab =',n_fv2)

Number of Words in Vocab = 337


In [40]:
                                              ## Prior Distribution ##
PD_w=[]
for x in vocab_2:
    PD_w.append(group.count(x)/len(group))         
print(' Sum of all prior probability=',sum(PD_w))

 Sum of all prior probability= 0.9999999999999966


In [45]:
                                      ## Class Conditional probability Of each word ## 

class_cond2 =np.zeros((n_fv2,n_fv2)) 
for i in range(n_fv2):                                   #class_word or row of class_cond2
    temp=[] 
    l=0
    for j in range(n_2) :                               #column for each class in class_cond2
        
        
        temp1 = filtering(df40.text[j].split())         #filters word
        temp1 = list(Counter(temp1))                    #removes duplicate

        
        if vocab_2[i] in temp1:                         #checks sentence if contains class_word 
            temp+=temp1                                 #adds all words of sentence to a list
            l+=1                                        #counts the number of sentence having class_word

    for k in range(n_fv2):                              #for assigning class cond prob to each word in vocab for class_word
        nk =temp.count(vocab_2[k]) 
        class_cond2[k,i]+=(nk+1)/(l+n_fv2)              # assigns prob with laplace smoothing

In [46]:
                                    ## Prediction of words fro sentences in 10.csv ##
            
for i in range(len(df10)):                                                 #test_sentence
    
    
    temp= filtering(df10.text[i].split())                                  #filters and makes of words in test sentence
    #temp = list(Counter(temp))

    #print(temp)                                                           #prints all the words cosidered in test sentence
    p1=np.ones((1,n_fv2))                                                  #p1 contains posterior prob of each class
    
    
    for j in temp:                                          #for each word in test sentence multiplying class condn prob to p1
        for k in range(n_fv2):
            if j==vocab_2[k]:
                p1 =np.multiply(class_cond2[k],p1)                       #if a word is not in vocab then it ignores the word
                break
       
                                                          #p1 now has product of class_cond prob of each word in test sentence 
        
    
    p1 = np.multiply(np.array(PD_w[0:]),p1)                                # product of prior prob to all class_cond prob
    p1 = p1/np.sum(p1,axis=1)                                              # converting p1 in probability
    
    
    print(i+1,df10.text[i], vocab_2[np.argmax(p1,axis=1)[0]])                           #prints results
    print('  Prob of chosen word =',np.max(p1,axis=1)[0])
    print('\n')
   

1 Eating healthy food is important for maintainng good _____ food
  Prob of chosen word = 0.542353618160035


2 Following a healthy diet will boost your _____ healthi
  Prob of chosen word = 0.503158001202429


3 Avoid eating chemical additives, added sugars in your diet. Switch to a healthy ____ food
  Prob of chosen word = 0.45006732953936474


4 Your diet should be rich of vitamins D,K, calcium and ______ diet
  Prob of chosen word = 0.3772323502596601


5 The healthier the food you eat, the better you’ll feel after a _____. food
  Prob of chosen word = 0.5754526085873335


6 Dehydration causes tiredness, low energy and head aches. Drink plenty of ______ energi
  Prob of chosen word = 0.2367365898130446


7 People with kidney disease should avoid eating high amounts of ______ food
  Prob of chosen word = 0.4126314428282017


8 We should avoid eating transfats. Eating healthy fats and dietary fibre can help us lose ____ food
  Prob of chosen word = 0.617931423559169


9 Mindless eati